In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-129142")
exp = Experiment(workspace=ws, name="quick-starts-ws-129142")
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-129142
Azure region: southcentralus
Subscription id: 8a56ebc5-caa7-4a43-b8ed-1c1895e46051
Resource group: aml-quickstarts-129142


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cluster_name="cpu-cluster"
try:
    new_cluster=ComputeTarget(workspace=ws, name="cpu-cluster")
    print("Existing cluster detected, make use of it!")
except ComputeTargetException:
    print("New compute cluster creation in progress...")
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',vm_priority='dedicated',min_node=None,max_nodes=4)
    new_cluster = ComputeTarget.create(ws, "new_cluster", compute_config)    
    new_cluster.wait_for_completion(show_output=True, max_nodes=4)         
    print("Cluster is ready")                                                             

Existing cluster detected, make use of it!


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
parameter_sampling = RandomParameterSampling( 
                    {
                        "--C":uniform(0.05,1.0),
                        "--max_iter":choice(25,50,75,100) 
                    }
)

#Specify a policy 
early_termination_policy = BanditPolicy(evaluation_interval=2,slack_factor=0.2,delay_evaluation=0)
 

# Create a SKLearn estimator for use with train.py
#import packages
from azureml.core import ScriptRunConfig
from azureml.core import Environment

training_script='train.py'

est=SKLearn(source_directory='.',entry_script=training_script,compute_target=new_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config =  HyperDriveConfig(estimator=est,
                              hyperparameter_sampling=parameter_sampling,
                              policy=early_termination_policy,
                              primary_metric_name="Accuracy",
                              primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                              max_total_runs=4,
                              max_concurrent_runs=4
                                     )

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

submit_hyperdrive = exp.submit(hyperdrive_config,show_output=True)
RunDetails(submit_hyperdrive).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [10]:
import joblib
# Get your best run.

### YOUR CODE HERE ###
best_run = submit_hyperdrive.get_best_run_by_primary_metric()
print(best_run)
best_run_metrics=best_run.get_metrics()
print(best_run_metrics)

# Save the model from the run.
best_run.download_file("outputs/hyperdrive_model.joblib","./outputs/hyperdrive_model.joblib")
hyperdrive_model = best_run.register_model(model_name = 'HyperDriveModel', model_path = 'outputs/hyperdrive_model.joblib')

Run(Experiment: quick-starts-ws-129142,
Id: HD_9c33e9ab-7d44-4298-91c8-25577bf61074_0,
Type: azureml.scriptrun,
Status: Completed)
{'Regularization Strength:': 0.7246866091952837, 'Max iterations:': 100, 'Accuracy': 0.9072837632776934}


In [85]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
src= "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds= TabularDatasetFactory.from_delimited_files(src)

In [86]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [87]:
#splitting the dataset
#import package
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test=train_test_split(x, y, train_size=0.8, test_size=0.2, random_state=42)

In [88]:
#Combining the training datasets
#import package
import pandas as pd

train_data=pd.concat((x_train, y_train), axis=1)

In [93]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='Accuracy',
    training_data=train_data,
    label_column_name='y',
    n_cross_validations=2,
    max_concurrent_iterations=4,
    featurization='auto')

In [94]:
# Retrieve and save your best automl model.
from azureml.core.experiment import Experiment

ws = Workspace.from_config()
experiment_name = 'automl-classification'

experiment = Experiment(ws, experiment_name)

automl_run = experiment.submit(automl_config, show_output=True)

ConfigException: ConfigException:
	Message: Invalid argument(s) 'primary_metric' specified. Supported value(s): 'precision_score_weighted, norm_macro_recall, AUC_weighted, average_precision_score_weighted, accuracy'.
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Invalid argument(s) 'primary_metric' specified. Supported value(s): 'precision_score_weighted, norm_macro_recall, AUC_weighted, average_precision_score_weighted, accuracy'.",
        "details_uri": "https://aka.ms/AutoMLConfig",
        "target": "primary_metric",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "ArgumentInvalid"
            }
        }
    }
}